In [275]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 5)

In [276]:
datafolder = 'input/'

## Qual é a rota mais comum das cargas?

In [277]:
shipments = pd.read_csv(datafolder+'shipments.csv')

#### Converte city_from e city_to para String para concatená-las criando uma coluna de índice

In [278]:
shipments['city_from'] = shipments['city_from'].astype(str)
shipments['city_to'] = shipments['city_to'].astype(str)

In [279]:
shipments['ROUTE'] = shipments['city_from'] + shipments['city_to']

#### Cria um df com apenas a coluna 'ROUTE' 

In [280]:
top_routes = pd.DataFrame(shipments['ROUTE'])

#### Agrupa as rotas e soma seus totais

In [281]:
top_routes = pd.DataFrame(top_routes.groupby(by=['ROUTE']).size()).reset_index()

In [282]:
top_routes.columns = ['ROUTE', 'SIZE']

In [283]:
top_routes.sort_values(by='SIZE', ascending=False).head(1)

,ROUTE,SIZE
26952,82689422,516


# Rota mais comum

In [284]:
shipments.ix[shipments['ROUTE']==\
                               top_routes.sort_values(by='SIZE', ascending=False).head(1).ix[:,'ROUTE'].values[0], \
                               ['city_from', 'city_to']].head(1)

,city_from,city_to
804,8268,9422


In [285]:
top_searched_routes.sort_values(by='SIZE', ascending=False).head(1).ix[:,'ROUTE'].values[0]

'9422-23.5505199-46.6333094'

## Qual é a rota procurada mais comum?

In [286]:
searches = pd.read_csv(datafolder+'searches.csv', dtype='unicode')

In [287]:
searches.columns

Index(['search_id', 'search_date', 'user_id', 'user_ip', 'truck_type_id',
       'body_type_id', 'truck_gps', 'city_from_id', 'city_to_id',
       'from_state_uf', 'to_state_uf', 'from_lat', 'from_lng', 'to_lat',
       'to_lng', 'search_source'],
      dtype='object')

In [288]:
for column in searches.columns:
    nulls = str(len(searches.ix[searches[column].isnull()]))
    print('Nulls in column ' + column + ' : ' + nulls)

Nulls in column search_id : 0
Nulls in column search_date : 0
Nulls in column user_id : 0
Nulls in column user_ip : 137828
Nulls in column truck_type_id : 0
Nulls in column body_type_id : 0
Nulls in column truck_gps : 0
Nulls in column city_from_id : 0
Nulls in column city_to_id : 0
Nulls in column from_state_uf : 789512
Nulls in column to_state_uf : 760145
Nulls in column from_lat : 81595
Nulls in column from_lng : 81595
Nulls in column to_lat : 508232
Nulls in column to_lng : 508232
Nulls in column search_source : 0


#### Como podemos ver acima, city_from e city_to não tem valores nulos

In [289]:
for column in searches.columns:
    nulls = str(len(searches.ix[searches[column] == 0]))
    print('0s in column ' + column + ' : ' + nulls)

0s in column search_id : 0
0s in column search_date : 0
0s in column user_id : 0
0s in column user_ip : 0
0s in column truck_type_id : 0
0s in column body_type_id : 0
0s in column truck_gps : 0
0s in column city_from_id : 0
0s in column city_to_id : 0
0s in column from_state_uf : 0
0s in column to_state_uf : 0
0s in column from_lat : 0
0s in column from_lng : 0
0s in column to_lat : 0
0s in column to_lng : 0
0s in column search_source : 0


#### Ou as colunas não são numéricas ou não temos 0 como valor. Vamos ver se temos 0 como String

In [290]:
for column in searches.columns:
    nulls = str(len(searches.ix[searches[column] == '0']))
    print('0s in column ' + column + ' : ' + nulls)

0s in column search_id : 0
0s in column search_date : 0
0s in column user_id : 109637
0s in column user_ip : 0
0s in column truck_type_id : 123797
0s in column body_type_id : 219643
0s in column truck_gps : 111003
0s in column city_from_id : 81595
0s in column city_to_id : 508232
0s in column from_state_uf : 0
0s in column to_state_uf : 5880
0s in column from_lat : 0
0s in column from_lng : 0
0s in column to_lat : 0
0s in column to_lng : 0
0s in column search_source : 374850


### Conclusões:
Os seguinges campos são <b>null</b> quando não tem informação disponível:<br>
Nulls in column from_state_uf : 789512<br>
Nulls in column to_state_uf : 760145<br>
Nulls in column from_lat : 81595<br>
Nulls in column from_lng : 81595<br>
Nulls in column to_lat : 508232<br>
Nulls in column to_lng : 508232<br>
Os seguintes campos são <b>'0'</b> quando não tem informação disponível:<br>
0s in column city_from_id : 81595<br>
0s in column city_to_id : 508232<br>
0s in column to_state_uf : 5880

In [291]:
resumo = pd.DataFrame(columns=['Pergunta', 'from_state_uf', 'city_from_id', 'from_lat', 'from_lng',\
                              'to_state_uf', 'city_to_id', 'to_lat', 'to_lng'])

In [292]:
resumo

,Pergunta,from_state_uf,city_from_id,from_lat,from_lng,to_state_uf,city_to_id,to_lat,to_lng


### Buscas só com origem

#### só com origem (neste caso so os campos city_from_id, from_lat e from_lng importam, os outros campos - city_to_id,  to_lat, to_lng, to_state_uf, são nulo sou string vazia),

#### Filtrar o df, removendo observações com to_lat e to_lng null, e removendo to_state_uf=='0' e city_to_id=='0'

In [293]:
searches_so_origem = searches.ix[(searches['to_lat'].isnull()) & (searches['to_lng'].isnull()) & \
                                (searches['to_state_uf']=='0') & (searches['city_to_id']=='0')]

In [294]:
searches_so_origem.shape

(5880, 16)

In [295]:
searches_so_origem['ROTA'] = searches_so_origem['city_from_id'] + searches_so_origem['from_lat'] + searches_so_origem['from_lng']

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [296]:
top_searched_routes = pd.DataFrame(searches_so_origem['ROTA'])
top_searched_routes = pd.DataFrame(top_searched_routes.groupby(by=['ROTA']).size()).reset_index()
top_searched_routes.columns = ['ROUTE', 'SIZE']
top_searched_routes.sort_values(by='SIZE', ascending=False).head(1)

,ROUTE,SIZE
245,9422-23.5505199-46.6333094,795


# Rota só com origem mais buscada

In [297]:
searches_so_origem.ix[searches_so_origem['ROTA']==\
                      top_searched_routes.sort_values(by='SIZE', ascending=False).head(1).ix[:,'ROUTE'].values[0],\
                      ['city_from_id', 'from_lat', 'from_lng']].head(1)

,city_from_id,from_lat,from_lng
548863,9422,-23.5505199,-46.6333094


In [298]:
aux = searches_so_origem.ix[searches_so_origem['ROTA']==\
                      top_searched_routes.sort_values(by='SIZE', ascending=False).head(1).ix[:,'ROUTE'].values[0],\
                      ['city_from_id', 'city_to_id', 'from_state_uf', 'to_state_uf', 'from_lat', 'from_lng', 'to_lat',\
                       'to_lng']].head(1)

aux['Pergunta'] = 'Rota só com origem mais buscada'
resumo = pd.concat([resumo, aux])
resumo.reset_index(inplace=True, drop=True)

In [299]:
resumo

,Pergunta,city_from_id,city_to_id,from_lat,from_lng,from_state_uf,to_lat,to_lng,to_state_uf
0,Rota só com origem mais buscada,9422,0,-23.5505199,-46.6333094,NaN,NaN,NaN,0


### Buscas com origem e destino

#### com origem e destino (neste caso importam os campos city_from_id, from_lat, from_lng, to_lat e to_lng),

In [300]:
searches_com_origem_destino = searches.ix[(searches['city_from_id'] !='0') & (searches['from_lat'].notnull()) & \
                                (searches['from_lng'].notnull()) & (searches['to_lat'].notnull()) \
                                          & (searches['to_lng'].notnull())]

In [301]:
searches_com_origem_destino.shape

(270092, 16)

In [302]:
searches_com_origem_destino.ix[:,'ROTA'] = searches_com_origem_destino.ix[:, 'city_from_id'] + \
searches_com_origem_destino.ix[:,'from_lat'] + searches_com_origem_destino.ix[:,'from_lng'] + \
searches_com_origem_destino.ix[:,'to_lat'] + searches_com_origem_destino.ix[:,'to_lng']


/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [303]:
top_searched_routes = pd.DataFrame(searches_com_origem_destino['ROTA'])
top_searched_routes = pd.DataFrame(top_searched_routes.groupby(by=['ROTA']).size()).reset_index()
top_searched_routes.columns = ['ROUTE', 'SIZE']
top_searched_routes.sort_values(by='SIZE', ascending=False).head(2)

,ROUTE,SIZE
35886,9422-23.5505199-46.6333094-23.5505199-46.6333094,6552
35797,9422-23.5505199-46.6333094-22.9082998-43.1970773,3926


# Rota com origem e destino mais buscada

In [304]:
searches_com_origem_destino.ix[searches_com_origem_destino['ROTA']==\
                               top_searched_routes.sort_values(by='SIZE', ascending=False).head(1).ix[:,'ROUTE'].values[0], \
                               ['city_from_id', 'from_lat', 'from_lng', 'to_lat', 'to_lng']].head(1)

,city_from_id,from_lat,from_lng,to_lat,to_lng
223,9422,-23.5505199,-46.6333094,-23.5505199,-46.6333094


In [305]:
aux = searches_com_origem_destino.ix[searches_com_origem_destino['ROTA']==\
                                     top_searched_routes.sort_values(by='SIZE', \
                                                                     ascending=False).head(1).ix[:,'ROUTE'].values[0],\
                                     ['city_from_id', 'city_to_id', 'from_state_uf', 'to_state_uf', 'from_lat', 'from_lng', \
                                      'to_lat', 'to_lng']].head(1)

aux['Pergunta'] = 'Rota com origem e destino mais buscada'
resumo = pd.concat([resumo, aux])
resumo.reset_index(inplace=True, drop=True)

#### O que notamos aqui é que a origem e o destino são o mesmo. Isso pode indicar a lat/long padrão usada para indicar o centro da cidade de SP, já que estão praticamente no marco zero da cidade:
<img src="https://raw.githubusercontent.com/guilhermemarson/cargox-test/master/marco-zero.png">

### Como ainda me falta conhecimento sobre a ferramenta, não posso assumir que o que foi encontrado acima é ou não um erro. Como ele não define uma rota propriamente dita, já que origem e destino são iguais, vou adicionar também a segunda rota mais buscada

In [306]:
searches_com_origem_destino.ix[searches_com_origem_destino['ROTA']==\
                               top_searched_routes.sort_values(by='SIZE', ascending=False).head(2).ix[:,'ROUTE'].values[1], \
                               ['city_from_id', 'from_lat', 'from_lng', 'to_lat', 'to_lng']].head(1)

,city_from_id,from_lat,from_lng,to_lat,to_lng
567,9422,-23.5505199,-46.6333094,-22.9082998,-43.1970773


In [307]:
aux = searches_com_origem_destino.ix[searches_com_origem_destino['ROTA']==\
                               top_searched_routes.sort_values(by='SIZE', ascending=False).head(2).ix[:,'ROUTE'].values[1], \
                               ['city_from_id', 'city_to_id', 'from_state_uf', 'to_state_uf', 'from_lat', 'from_lng', \
                                'to_lat', 'to_lng']].head(1)

aux['Pergunta'] = 'Segunda rota com origem e destino mais buscada'
resumo = pd.concat([resumo, aux])
resumo.reset_index(inplace=True, drop=True)

<img src="https://raw.githubusercontent.com/guilhermemarson/cargox-test/master/SP-RJ.png">

### Buscas com origem e estado de destino

#### com origem e estado de destino (aqui os campos que importam seriam city_from_id,from_lat,from_lng,to_state_uf, os outros campos serão nulos ou string vazias)

In [308]:
orig_est_dest = searches.ix[(searches['city_to_id'] =='0') & (searches['to_lat'].isnull()) & \
                                (searches['to_lng'].isnull()) & (searches['city_from_id']!=0) \
                           & (searches['from_lat'].notnull()) & (searches['from_lng'].notnull()) \
                           & (searches['to_state_uf']!='0')]
#& (searches['from_state_uf'].isnull())

In [309]:
orig_est_dest.ix[orig_est_dest.from_state_uf.isnull()].shape

(433159, 16)

In [310]:
orig_est_dest.shape

(433159, 16)

#### Eu tinha dúvidas se poderia considerar o from_state_uf vazio ou se poderia ter casos em que o estado de origem estaria definido, mas após o estudo acima, consegui concluir que ele é sempre vazio neste caso 

In [311]:
orig_est_dest.ix[:,'ROTA'] = orig_est_dest.ix[:, 'city_from_id'] + \
orig_est_dest.ix[:,'from_lat'] + orig_est_dest.ix[:,'from_lng'] + \
orig_est_dest.ix[:,'to_state_uf'] 

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [312]:
top_searched_routes = pd.DataFrame(orig_est_dest['ROTA'])
top_searched_routes = pd.DataFrame(top_searched_routes.groupby(by=['ROTA']).size()).reset_index()
top_searched_routes.columns = ['ROUTE', 'SIZE']
top_searched_routes.sort_values(by='SIZE', ascending=False).head(1)

,ROUTE,SIZE
2094,9422-23.5505199-46.6333094SP,742


# Rota com origem e estado de destino mais buscada

In [313]:
orig_est_dest.ix[orig_est_dest['ROTA']==\
                 top_searched_routes.sort_values(by='SIZE', ascending=False).head(1).ix[:,'ROUTE'].values[0], \
                               ['city_from_id', 'from_lat', 'from_lng', 'to_state_uf']].head(1)

,city_from_id,from_lat,from_lng,to_state_uf
542649,9422,-23.5505199,-46.6333094,SP


In [314]:
aux = orig_est_dest.ix[orig_est_dest['ROTA']==\
                 top_searched_routes.sort_values(by='SIZE', ascending=False).head(1).ix[:,'ROUTE'].values[0], \
                               ['city_from_id', 'city_to_id', 'from_state_uf', 'to_state_uf', 'from_lat', 'from_lng', \
                                'to_lat', 'to_lng']].head(1)

aux['Pergunta'] = 'Rota com origem e estado de destino mais buscada'
resumo = pd.concat([resumo, aux])
resumo.reset_index(inplace=True, drop=True)

### O dataframe abaixo resume a segunda parte da primeira pergunta. 
### Separei a pergunta de rota procurada mais comum nos 3 tipos de busca documentados no arquivo
### O segundo tipo de busca foi quebrado em duas, pois a primeira tinha origem e destino iguais

In [315]:
resumo

,Pergunta,city_from_id,city_to_id,from_lat,from_lng,from_state_uf,to_lat,to_lng,to_state_uf
0,Rota só com origem mais buscada,9422,0,-23.5505199,-46.6333094,NaN,NaN,NaN,0
1,Rota com origem e destino mais buscada,9422,9422,-23.5505199,-46.6333094,NaN,-23.5505199,-46.6333094,NaN
2,Segunda rota com origem e destino mais buscada,9422,6861,-23.5505199,-46.6333094,NaN,-22.9082998,-43.1970773,NaN
3,Rota com origem e estado de destino mais buscada,9422,0,-23.5505199,-46.6333094,NaN,NaN,NaN,SP
